In [1]:
import os
import sys
import random

import numpy as np
import cv2
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

In [2]:
class DataGen(keras.utils.Sequence):
    def __init__(self, ids, path, batch_size=8, image_size=128):
        self.ids = ids
        self.path = path
        self.batch_size = batch_size
        self.image_size = image_size
        self.on_epoch_end()
        
    def __load__(self, id_name):
        # will be refactored, cuz looks like wtf
        prefix = id_name[:22]
        image_path = os.path.join(self.path, "inpaint/", "result-"+prefix+"-"+id_name) + ".png"
        mask_path = os.path.join(self.path, "masks/", id_name) + ".jpg"
        
        image = cv2.imread(image_path, 1)
        image = cv2.resize(image, (self.image_size, self.image_size))
        mask = np.zeros((self.image_size, self.image_size, 1))
        
        mask_im = cv2.imread(mask_path, 0)
        mask_im = cv2.bitwise_not(mask_im)
        mask_im = cv2.resize(mask_im, (self.image_size, self.image_size))
        
        mask = np.expand_dims(mask_im , axis=-1)
        
        image = image / 255.0
        mask = mask / 255.0
        
        return image, mask
    
    def __getitem__(self, index):
        if(index + 1)*self.batch_size > len(self.ids):
            self.batch_size = len(self.ids) - index*self.batch_size
            
        files_batch = self.ids[index*self.batch_size : (index + 1)*self.batch_size]
        
        image = []
        mask = []
        
        for id_name in files_batch:
            _img, _mask = self.__load__(id_name)
            image.append(_img)
            mask.append(_mask)
            
            
        image = np.array(image)
        mask = np.array(mask)    
            
        return image, mask
    
    def on_epoch_end(self):
        pass
    
    def __len__(self):
        return int(np.ceil(len(self.ids)/float(self.batch_size)))

In [3]:
image_size = 256
train_path = "dataset/DFNet_ellipse/"
epochs = 10
batch_size = 5
val_data_size = 400

flist_path = train_path + "masks/flist_mask"
train_ids = open(flist_path).read().splitlines()
valid_ids = train_ids[:val_data_size]
all_ids = train_ids
train_ids = train_ids[val_data_size:]

In [4]:
def down_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    p = keras.layers.MaxPool2D((2, 2), (2, 2))(c)
    return c, p

def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    us = keras.layers.UpSampling2D((2, 2))(x)
    concat = keras.layers.Concatenate()([us, skip])
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(concat)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

def bottleneck(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

In [5]:
def UNet():
    f = [16, 32, 64, 128, 256, 512]
    inputs = keras.layers.Input((image_size, image_size, 3))
    
    p0 = inputs
    c1, p1 = down_block(p0, f[0]) #256 -> 128
    c2, p2 = down_block(p1, f[1]) #128 -> 64
    c3, p3 = down_block(p2, f[2]) #64 -> 32
    c4, p4 = down_block(p3, f[3]) #32 -> 16
    c5, p5 = down_block(p4, f[4]) #16->8
    
    bn = bottleneck(p5, f[5])
    
    u1 = up_block(bn, c5, f[4]) #8 -> 16
    u2 = up_block(u1, c4, f[3]) #16 -> 32
    u3 = up_block(u2, c3, f[2]) #32 -> 64
    u4 = up_block(u3, c2, f[1]) #64 -> 128
    u5 = up_block(u4, c1, f[0]) #128 -> 256
    
    outputs = keras.layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(u5)
    model = keras.models.Model(inputs, outputs)
    return model

In [7]:
model = UNet()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_23 (Conv2D)              (None, 256, 256, 16) 448         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_24 (Conv2D)              (None, 256, 256, 16) 2320        conv2d_23[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_5 (MaxPooling2D)  (None, 128, 128, 16) 0           conv2d_24[0][0]                  
____________________________________________________________________________________________

Trainable params: 7,862,401
Non-trainable params: 0
__________________________________________________________________________________________________


In [8]:
train_gen = DataGen(train_ids, train_path, image_size=image_size, batch_size=batch_size)
valid_gen = DataGen(valid_ids, train_path, image_size=image_size, batch_size=batch_size)

train_steps = len(train_ids)//batch_size
valid_steps = len(valid_ids)//batch_size

model.fit_generator(train_gen, validation_data=valid_gen, steps_per_epoch=train_steps, validation_steps=valid_steps, epochs=epochs)

Epoch 1/10
165/165 [==============================] - 252s 2s/step - loss: 0.2529 - acc: 0.9363 - val_loss: 0.1648 - val_acc: 0.9448
Epoch 2/10
165/165 [==============================] - 213s 1s/step - loss: 0.2831 - acc: 0.9445 - val_loss: 0.2109 - val_acc: 0.9448
Epoch 3/10
165/165 [==============================] - 213s 1s/step - loss: 0.2525 - acc: 0.9441 - val_loss: 0.1987 - val_acc: 0.9448
Epoch 4/10
165/165 [==============================] - 213s 1s/step - loss: 0.1853 - acc: 0.9446 - val_loss: 0.1762 - val_acc: 0.9448
Epoch 5/10
165/165 [==============================] - 214s 1s/step - loss: 0.1823 - acc: 0.9447 - val_loss: 0.1780 - val_acc: 0.9448
Epoch 6/10
165/165 [==============================] - 211s 1s/step - loss: 0.1769 - acc: 0.9444 - val_loss: 0.1712 - val_acc: 0.9448
Epoch 7/10
165/165 [==============================] - 211s 1s/step - loss: 0.1703 - acc: 0.9448 - val_loss: 0.1693 - val_acc: 0.9448
Epoch 8/10
165/165 [==============================] - 212s 1s/step - 

In [9]:
model.save_weights("DFNet_UnetW_5_10_ellipse.h5")